## Data Cleaning - Alternative
----
In this interactive notebook, we explore the process of cleaning up daily summary data downloaded en masse through the AQS website. The website provides ZIP archives of CSV files for each of the parameters of Criteria gasses, which are all contained in the AQI defined pollutants class of parameters. I have collected ZIP files with daily summary data for each of Carbon Monoxide (CO), Ozone (O3), Sulfur Dioxide (SO2), and Nitrogen Dioxide (NO2) from 2010 to 2018.

Let's start by importing our required libraries.

In [1]:
import pandas as pd
from pyaqs import AQSFetcher

In [2]:
aqs_fetcher = AQSFetcher('bbjornstad.flatiron@gmail.com', 'ochrefox21')

In [3]:
param_codes = aqs_fetcher.get_parameter_list_by_class('AQI POLLUTANTS')
state_codes = aqs_fetcher.get_state_codes()

In [4]:
data_folder = 'agg_data_aqs/'

In [5]:
param_codes_gasses = param_codes.iloc[0:4,:]
param_codes_gasses

,code,parameter_description
0,42101,Carbon monoxide
1,42401,Sulfur dioxide
2,42602,Nitrogen dioxide (NO2)
3,44201,Ozone


Let's store a list holding all of the years for which we have downloaded daily summary data for each of the above four gas parameters.

In [6]:
date_range = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

Now, with this, we can import and clean up our datasets.

In [7]:
cols_to_drop = [
    'POC',
    'Latitude',
    'Longitude',
    'Datum',
    'Pollutant Standard',
    'Event Type',
    'Observation Count',
    'Observation Percent',
    '1st Max Value',
    '1st Max Hour',
    'Method Code',
    'Method Name',
    'Address',
    'CBSA Name',
    'Date of Last Change',
    'Local Site Name']

In [8]:
def import_aggregate_data(param_code, year):
    path_stub = f'/daily_{param_code}_{year}.zip'
    path = data_folder+path_stub
    df = pd.read_csv(path)
    df.drop(columns = cols_to_drop, inplace=True)
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df.date_local = pd.to_datetime(df.date_local)
    return df

In [9]:
all_data_raw_agg_trimmed = pd.DataFrame()

for param_code in param_codes_gasses.code:
    print(param_code)
    all_param_df = pd.DataFrame()
    for year in date_range:
        yearly_param_df = import_aggregate_data(param_code, year)
        all_param_df = pd.concat([all_param_df, yearly_param_df])
        
    all_param_df.to_csv(f'cleaned_data/{param_code}_daily.csv')
    
    all_data_raw_agg_trimmed = pd.concat([all_data_raw_agg_trimmed, all_param_df])

42101
42401
42602


//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


44201


In [12]:
all_data_raw_agg_trimmed.shape

(9710154, 13)

In [16]:
all_data_raw_agg_trimmed.head()

,state_code,county_code,site_num,parameter_code,parameter_name,sample_duration,date_local,units_of_measure,arithmetic_mean,aqi,state_name,county_name,city_name
0,1,73,28,42101,Carbon monoxide,1 HOUR,2010-01-01,Parts per million,0.470833,NaN,Alabama,Jefferson,Birmingham
1,1,73,28,42101,Carbon monoxide,1 HOUR,2010-01-02,Parts per million,0.479167,NaN,Alabama,Jefferson,Birmingham
2,1,73,28,42101,Carbon monoxide,1 HOUR,2010-01-03,Parts per million,0.462500,NaN,Alabama,Jefferson,Birmingham
3,1,73,28,42101,Carbon monoxide,1 HOUR,2010-01-04,Parts per million,0.579167,NaN,Alabama,Jefferson,Birmingham
4,1,73,28,42101,Carbon monoxide,1 HOUR,2010-01-05,Parts per million,0.582609,NaN,Alabama,Jefferson,Birmingham


In [17]:
all_data_raw_agg_trimmed.sample_duration.value_counts()

1 HOUR                     3760545
8-HR RUN AVG BEGIN HOUR    3502101
3-HR BLK AVG               1480418
8-HR RUN AVG END HOUR       967090
Name: sample_duration, dtype: int64

In [18]:
all_data_raw_agg_trimmed.date_local.value_counts()

2016-05-26    3348
2016-05-24    3312
2016-07-22    3308
2016-07-20    3303
2016-05-23    3289
              ... 
2010-12-25    2481
2010-11-26    2473
2010-12-26    2473
2010-11-27    2471
2010-11-28    2469
Name: date_local, Length: 3287, dtype: int64